# Multi-Carrier

Hospital with CHP producing both electricity and heat.

This notebook introduces:

- **Multiple energy carriers**: Electricity, heat, and gas in one system
- **CHP (Cogeneration)**: Equipment producing multiple outputs
- **Electricity market**: Buying and selling to the grid
- **Carrier colors**: Visual distinction between energy types

## Setup

In [1]:
import pandas as pd
import xarray as xr

import flixopt as fx

fx.CONFIG.notebook()

flixopt.config.CONFIG

## System Description

The hospital energy system:

```
                    Grid Buy ──►
                                 [Electricity] ──► Hospital Elec. Load
                    Grid Sell ◄──     ▲
                                      │
     Gas Grid ──► [Gas] ──► CHP ──────┘
                    │         │
                    │         ▼
                    │     [Heat] ──► Hospital Heat Load
                    │         ▲
                    └──► Boiler
```

**Equipment:**
- **CHP**: 200 kW electrical, ~250 kW thermal (η_el=40%, η_th=50%)
- **Gas Boiler**: 400 kW thermal backup
- **Grid**: Buy electricity at variable prices, sell at lower prices

## Define Time Horizon and Demand Profiles

In [2]:
from data.tutorial_data import get_multicarrier_data

data = get_multicarrier_data()
timesteps = data['timesteps']
electricity_demand = data['electricity_demand']
heat_demand = data['heat_demand']
elec_buy_price = data['elec_buy_price']
elec_sell_price = data['elec_sell_price']
gas_price = data['gas_price']

In [3]:
# Visualize demands and prices with plotly
profiles = xr.Dataset(
    {
        'Electricity Demand [kW]': xr.DataArray(electricity_demand, dims=['time'], coords={'time': timesteps}),
        'Heat Demand [kW]': xr.DataArray(heat_demand, dims=['time'], coords={'time': timesteps}),
        'Elec. Buy Price [EUR/kWh]': xr.DataArray(elec_buy_price, dims=['time'], coords={'time': timesteps}),
    }
)
profiles.plotly.line(x='time', title='Hospital Energy Profiles', height=300)

## Build the Multi-Carrier System

In [4]:
flow_system = fx.FlowSystem(timesteps, name='With CHP')
flow_system.add_carriers(
    fx.Carrier('gas', '#3498db', 'kW'),
    fx.Carrier('electricity', '#f1c40f', 'kW'),
    fx.Carrier('heat', '#e74c3c', 'kW'),
)
flow_system.add_elements(
    # === Buses with carriers for visual distinction ===
    fx.Bus('Electricity', carrier='electricity'),
    fx.Bus('Heat', carrier='heat'),
    fx.Bus('Gas', carrier='gas'),
    # === Effects ===
    fx.Effect('costs', '€', 'Total Costs', is_standard=True, is_objective=True),
    fx.Effect('CO2', 'kg', 'CO2 Emissions'),  # Track emissions too
    # === Gas Supply ===
    fx.Source(
        'GasGrid',
        outputs=[
            fx.Flow(
                'Gas',
                bus='Gas',
                size=1000,
                effects_per_flow_hour={'costs': gas_price, 'CO2': 0.2},  # Gas: 0.2 kg CO2/kWh
            )
        ],
    ),
    # === Electricity Grid (buy) ===
    fx.Source(
        'GridBuy',
        outputs=[
            fx.Flow(
                'Electricity',
                bus='Electricity',
                size=500,
                effects_per_flow_hour={'costs': elec_buy_price, 'CO2': 0.4},  # Grid: 0.4 kg CO2/kWh
            )
        ],
    ),
    # === Electricity Grid (sell) - negative cost = revenue ===
    fx.Sink(
        'GridSell',
        inputs=[
            fx.Flow(
                'Electricity',
                bus='Electricity',
                size=200,
                effects_per_flow_hour={'costs': -elec_sell_price},  # Negative = income
            )
        ],
    ),
    # === CHP Unit (Combined Heat and Power) ===
    fx.linear_converters.CHP(
        'CHP',
        electrical_efficiency=0.40,  # 40% to electricity
        thermal_efficiency=0.50,  # 50% to heat (total: 90%)
        status_parameters=fx.StatusParameters(
            effects_per_startup={'costs': 30},
            min_uptime=3,
        ),
        electrical_flow=fx.Flow('P_el', bus='Electricity', size=200),
        thermal_flow=fx.Flow('Q_th', bus='Heat', size=250),
        fuel_flow=fx.Flow(
            'Q_fuel',
            bus='Gas',
            size=500,
            relative_minimum=0.4,  # Min 40% load
        ),
    ),
    # === Gas Boiler (heat only) ===
    fx.linear_converters.Boiler(
        'Boiler',
        thermal_efficiency=0.92,
        thermal_flow=fx.Flow('Q_th', bus='Heat', size=400),
        fuel_flow=fx.Flow('Q_fuel', bus='Gas'),
    ),
    # === Hospital Loads ===
    fx.Sink(
        'HospitalElec',
        inputs=[fx.Flow('Load', bus='Electricity', size=1, fixed_relative_profile=electricity_demand)],
    ),
    fx.Sink(
        'HospitalHeat',
        inputs=[fx.Flow('Load', bus='Heat', size=1, fixed_relative_profile=heat_demand)],
    ),
)

## Run Optimization

In [5]:
flow_system.optimize(fx.solvers.HighsSolver(mip_gap=0.01));

## Analyze Results

### Electricity Balance

In [6]:
flow_system.statistics.plot.balance('Electricity')

PlotResult(data=<xarray.Dataset> Size: 7kB
Dimensions:                (time: 169)
Coordinates:
  * time                   (time) datetime64[ns] 1kB 2024-02-05 ... 2024-02-12
Data variables:
    GridBuy(Electricity)   (time) float64 1kB -0.0 -0.0 -19.24 ... -41.05 nan
    CHP(P_el)              (time) float64 1kB -192.4 -178.3 ... -120.0 nan
    GridSell(Electricity)  (time) float64 1kB 52.41 35.76 0.0 ... 31.05 0.0 nan
    HospitalElec(Load)     (time) float64 1kB 140.0 142.5 159.3 ... 161.1 nan, figure=Figure({
    'data': [{'fillcolor': '#EF553B',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f} kW<extra></extra>',
              'legendgroup': 'GridBuy(Electricity)',
              'line': {'color': '#EF553B', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'GridBuy(Electricity)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'negative',
              'type': 'scatter',
              'x': array(['2024-02-05T00:00:00.000000000', '2024-02-05T01:00:00.000000000',
                          '2024-02-05T02:00:00.000000000', '2024-02-05T03:00:00.000000000',
                          '2024-02-05T04:00:00.000000000', '2024-02-05T05:00:00.000000000',
                          '2024-02-05T06:00:00.000000000', '2024-02-05T07:00:00.000000000',
                          '2024-02-05T08:00:00.000000000', '2024-02-05T09:00:00.000000000',
                          '2024-02-05T10:00:00.000000000', '2024-02-05T11:00:00.000000000',
                          '2024-02-05T12:00:00.000000000', '2024-02-05T13:00:00.000000000',
                          '2024-02-05T14:00:00.000000000', '2024-02-05T15:00:00.000000000',
                          '2024-02-05T16:00:00.000000000', '2024-02-05T17:00:00.000000000',
                          '2024-02-05T18:00:00.000000000', '2024-02-05T19:00:00.000000000',
                          '2024-02-05T20:00:00.000000000', '2024-02-05T21:00:00.000000000',
                          '2024-02-05T22:00:00.000000000', '2024-02-05T23:00:00.000000000',
                          '2024-02-06T00:00:00.000000000', '2024-02-06T01:00:00.000000000',
                          '2024-02-06T02:00:00.000000000', '2024-02-06T03:00:00.000000000',
                          '2024-02-06T04:00:00.000000000', '2024-02-06T05:00:00.000000000',
                          '2024-02-06T06:00:00.000000000', '2024-02-06T07:00:00.000000000',
                          '2024-02-06T08:00:00.000000000', '2024-02-06T09:00:00.000000000',
                          '2024-02-06T10:00:00.000000000', '2024-02-06T11:00:00.000000000',
                          '2024-02-06T12:00:00.000000000', '2024-02-06T13:00:00.000000000',
                          '2024-02-06T14:00:00.000000000', '2024-02-06T15:00:00.000000000',
                          '2024-02-06T16:00:00.000000000', '2024-02-06T17:00:00.000000000',
                          '2024-02-06T18:00:00.000000000', '2024-02-06T19:00:00.000000000',
                          '2024-02-06T20:00:00.000000000', '2024-02-06T21:00:00.000000000',
                          '2024-02-06T22:00:00.000000000', '2024-02-06T23:00:00.000000000',
                          '2024-02-07T00:00:00.000000000', '2024-02-07T01:00:00.000000000',
                          '2024-02-07T02:00:00.000000000', '2024-02-07T03:00:00.000000000',
                          '2024-02-07T04:00:00.000000000', '2024-02-07T05:00:00.000000000',
                          '2024-02-07T06:00:00.000000000', '2024-02-07T07:00:00.000000000',
                          '2024-02-07T08:00:00.000000000', '2024-02-07T09:00:00.000000000',
                          '2024-02-07T10:00:00.000000000', '2024-02-07T11:00:00.000000000',
                          '2024-02-07T12:00:00.000000000', '2024-02-07T13:00:00.000000000',
                          '2024-02-07T14:00:00.000000000', '2024-02-07T15:

### Heat Balance

In [7]:
flow_system.statistics.plot.balance('Heat')

PlotResult(data=<xarray.Dataset> Size: 5kB
Dimensions:             (time: 169)
Coordinates:
  * time                (time) datetime64[ns] 1kB 2024-02-05 ... 2024-02-12
Data variables:
    CHP(Q_th)           (time) float64 1kB -240.5 -222.9 -175.1 ... -150.0 nan
    Boiler(Q_th)        (time) float64 1kB -2.842e-14 -0.0 -0.0 ... -0.0 nan
    HospitalHeat(Load)  (time) float64 1kB 240.5 222.9 175.1 ... 219.0 150.0 nan, figure=Figure({
    'data': [{'fillcolor': '#AB63FA',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f} kW<extra></extra>',
              'legendgroup': 'CHP(Q_th)',
              'line': {'color': '#AB63FA', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'CHP(Q_th)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'negative',
              'type': 'scatter',
              'x': array(['2024-02-05T00:00:00.000000000', '2024-02-05T01:00:00.000000000',
                          '2024-02-05T02:00:00.000000000', '2024-02-05T03:00:00.000000000',
                          '2024-02-05T04:00:00.000000000', '2024-02-05T05:00:00.000000000',
                          '2024-02-05T06:00:00.000000000', '2024-02-05T07:00:00.000000000',
                          '2024-02-05T08:00:00.000000000', '2024-02-05T09:00:00.000000000',
                          '2024-02-05T10:00:00.000000000', '2024-02-05T11:00:00.000000000',
                          '2024-02-05T12:00:00.000000000', '2024-02-05T13:00:00.000000000',
                          '2024-02-05T14:00:00.000000000', '2024-02-05T15:00:00.000000000',
                          '2024-02-05T16:00:00.000000000', '2024-02-05T17:00:00.000000000',
                          '2024-02-05T18:00:00.000000000', '2024-02-05T19:00:00.000000000',
                          '2024-02-05T20:00:00.000000000', '2024-02-05T21:00:00.000000000',
                          '2024-02-05T22:00:00.000000000', '2024-02-05T23:00:00.000000000',
                          '2024-02-06T00:00:00.000000000', '2024-02-06T01:00:00.000000000',
                          '2024-02-06T02:00:00.000000000', '2024-02-06T03:00:00.000000000',
                          '2024-02-06T04:00:00.000000000', '2024-02-06T05:00:00.000000000',
                          '2024-02-06T06:00:00.000000000', '2024-02-06T07:00:00.000000000',
                          '2024-02-06T08:00:00.000000000', '2024-02-06T09:00:00.000000000',
                          '2024-02-06T10:00:00.000000000', '2024-02-06T11:00:00.000000000',
                          '2024-02-06T12:00:00.000000000', '2024-02-06T13:00:00.000000000',
                          '2024-02-06T14:00:00.000000000', '2024-02-06T15:00:00.000000000',
                          '2024-02-06T16:00:00.000000000', '2024-02-06T17:00:00.000000000',
                          '2024-02-06T18:00:00.000000000', '2024-02-06T19:00:00.000000000',
                          '2024-02-06T20:00:00.000000000', '2024-02-06T21:00:00.000000000',
                          '2024-02-06T22:00:00.000000000', '2024-02-06T23:00:00.000000000',
                          '2024-02-07T00:00:00.000000000', '2024-02-07T01:00:00.000000000',
                          '2024-02-07T02:00:00.000000000', '2024-02-07T03:00:00.000000000',
                          '2024-02-07T04:00:00.000000000', '2024-02-07T05:00:00.000000000',
                          '2024-02-07T06:00:00.000000000', '2024-02-07T07:00:00.000000000',
                          '2024-02-07T08:00:00.000000000', '2024-02-07T09:00:00.000000000',
                          '2024-02-07T10:00:00.000000000', '2024-02-07T11:00:00.000000000',
                          '2024-02-07T12:00:00.000000000', '2024-02-07T13:00:00.000000000',
                          '2024-02-07T14:00:00.000000000', '2024-02-07T15:00:00.000000000',
                          '2024-02-07T16:00:00.000000000', '2024-02-07T17:00:00.0000

### Gas Balance

In [8]:
flow_system.statistics.plot.balance('Gas')

PlotResult(data=<xarray.Dataset> Size: 5kB
Dimensions:         (time: 169)
Coordinates:
  * time            (time) datetime64[ns] 1kB 2024-02-05 ... 2024-02-12
Data variables:
    GasGrid(Gas)    (time) float64 1kB -481.0 -445.7 -350.1 ... -300.0 nan
    CHP(Q_fuel)     (time) float64 1kB 481.0 445.7 350.1 ... 437.9 300.0 nan
    Boiler(Q_fuel)  (time) float64 1kB 3.089e-14 0.0 0.0 0.0 ... 0.0 0.0 nan, figure=Figure({
    'data': [{'fillcolor': '#636EFA',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f} kW<extra></extra>',
              'legendgroup': 'GasGrid(Gas)',
              'line': {'color': '#636EFA', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'GasGrid(Gas)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'negative',
              'type': 'scatter',
              'x': array(['2024-02-05T00:00:00.000000000', '2024-02-05T01:00:00.000000000',
                          '2024-02-05T02:00:00.000000000', '2024-02-05T03:00:00.000000000',
                          '2024-02-05T04:00:00.000000000', '2024-02-05T05:00:00.000000000',
                          '2024-02-05T06:00:00.000000000', '2024-02-05T07:00:00.000000000',
                          '2024-02-05T08:00:00.000000000', '2024-02-05T09:00:00.000000000',
                          '2024-02-05T10:00:00.000000000', '2024-02-05T11:00:00.000000000',
                          '2024-02-05T12:00:00.000000000', '2024-02-05T13:00:00.000000000',
                          '2024-02-05T14:00:00.000000000', '2024-02-05T15:00:00.000000000',
                          '2024-02-05T16:00:00.000000000', '2024-02-05T17:00:00.000000000',
                          '2024-02-05T18:00:00.000000000', '2024-02-05T19:00:00.000000000',
                          '2024-02-05T20:00:00.000000000', '2024-02-05T21:00:00.000000000',
                          '2024-02-05T22:00:00.000000000', '2024-02-05T23:00:00.000000000',
                          '2024-02-06T00:00:00.000000000', '2024-02-06T01:00:00.000000000',
                          '2024-02-06T02:00:00.000000000', '2024-02-06T03:00:00.000000000',
                          '2024-02-06T04:00:00.000000000', '2024-02-06T05:00:00.000000000',
                          '2024-02-06T06:00:00.000000000', '2024-02-06T07:00:00.000000000',
                          '2024-02-06T08:00:00.000000000', '2024-02-06T09:00:00.000000000',
                          '2024-02-06T10:00:00.000000000', '2024-02-06T11:00:00.000000000',
                          '2024-02-06T12:00:00.000000000', '2024-02-06T13:00:00.000000000',
                          '2024-02-06T14:00:00.000000000', '2024-02-06T15:00:00.000000000',
                          '2024-02-06T16:00:00.000000000', '2024-02-06T17:00:00.000000000',
                          '2024-02-06T18:00:00.000000000', '2024-02-06T19:00:00.000000000',
                          '2024-02-06T20:00:00.000000000', '2024-02-06T21:00:00.000000000',
                          '2024-02-06T22:00:00.000000000', '2024-02-06T23:00:00.000000000',
                          '2024-02-07T00:00:00.000000000', '2024-02-07T01:00:00.000000000',
                          '2024-02-07T02:00:00.000000000', '2024-02-07T03:00:00.000000000',
                          '2024-02-07T04:00:00.000000000', '2024-02-07T05:00:00.000000000',
                          '2024-02-07T06:00:00.000000000', '2024-02-07T07:00:00.000000000',
                          '2024-02-07T08:00:00.000000000', '2024-02-07T09:00:00.000000000',
                          '2024-02-07T10:00:00.000000000', '2024-02-07T11:00:00.000000000',
                          '2024-02-07T12:00:00.000000000', '2024-02-07T13:00:00.000000000',
                          '2024-02-07T14:00:00.000000000', '2024-02-07T15:00:00.000000000',
                          '2024-02-07T16:00:00.000000000', '2024-02-07T17:00:00.000000000',
  

### CHP Operation Pattern

In [9]:
flow_system.statistics.plot.heatmap('CHP(P_el)')

PlotResult(data=<xarray.Dataset> Size: 2kB
Dimensions:    (timeframe: 8, timestep: 24)
Coordinates:
  * timeframe  (timeframe) object 64B '2024-02-05' '2024-02-06' ... '2024-02-12'
  * timestep   (timestep) object 192B '00:00' '01:00' ... '22:00' '23:00'
Data variables:
    value      (timestep, timeframe) float64 2kB 192.4 161.8 167.9 ... 120.0 nan, figure=Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'timeframe: %{x}<br>timestep: %{y}<br>CHP(P_el)|flow_rate: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': array(['2024-02-05', '2024-02-06', '2024-02-07', '2024-02-08', '2024-02-09',
                          '2024-02-10', '2024-02-11', '2024-02-12'], dtype=object),
              'xaxis': 'x',
              'y': array(['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00',
                          '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00',
                          '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'],
                         dtype=object),
              'yaxis': 'y',
              'z': {'bdata': ('XDVrBksMaEDKS84bIThkQE6pKcfy+2' ... '9YxOQCY0AAAAAAAABeQAAAAAAAAPh/'),
                    'dtype': 'f8',
                    'shape': '24, 8'}}],
    'layout': {'coloraxis': {'cmax': 200.0,
                             'cmin': 120.0,
                             'colorbar': {'title': {'text': 'CHP(P_el)|flow_rate'}},
                             'colorscale': [[0.0, '#30123b'],
                                            [0.07142857142857142, '#4145ab'],
                                            [0.14285714285714285, '#4675ed'],
                                            [0.21428571428571427, '#39a2fc'],
                                            [0.2857142857142857, '#1bcfd4'],
                                            [0.35714285714285715, '#24eca6'],
                                            [0.42857142857142855, '#61fc6c'], [0.5,
                                            '#a4fc3b'], [0.5714285714285714,
                                            '#d1e834'], [0.6428571428571429,
                                            '#f3c63a'], [0.7142857142857143,
                                            '#fe9b2d'], [0.7857142857142857,
                                            '#f36315'], [0.8571428571428571,
                                            '#d93806'], [0.9285714285714286,
                                            '#b11901'], [1.0, '#7a0402']]},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'timeframe'}},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'domain': [0.0, 1.0], 'title': {'text': 'timestep'}}}
}))

### Cost and Emissions Summary

In [10]:
# Energy flows
flow_rates = flow_system.statistics.flow_rates
grid_buy = flow_rates['GridBuy(Electricity)'].sum().item()
grid_sell = flow_rates['GridSell(Electricity)'].sum().item()
chp_elec = flow_rates['CHP(P_el)'].sum().item()
chp_heat = flow_rates['CHP(Q_th)'].sum().item()
boiler_heat = flow_rates['Boiler(Q_th)'].sum().item()

total_elec = electricity_demand.sum()
total_heat = heat_demand.sum()

pd.DataFrame(
    {
        'CHP Electricity [kWh]': chp_elec,
        'CHP Electricity [%]': chp_elec / total_elec * 100,
        'Grid Buy [kWh]': grid_buy,
        'Grid Sell [kWh]': grid_sell,
        'CHP Heat [kWh]': chp_heat,
        'CHP Heat [%]': chp_heat / total_heat * 100,
        'Boiler Heat [kWh]': boiler_heat,
        'Total Costs [EUR]': flow_system.solution['costs'].item(),
        'Total CO2 [kg]': flow_system.solution['CO2'].item(),
    },
    index=['Value'],
).T

,Value
CHP Electricity [kWh],31304.424640
CHP Electricity [%],102.063863
Grid Buy [kWh],2475.425725
Grid Sell [kWh],3108.441560
CHP Heat [kWh],39130.530800
CHP Heat [%],88.580014
Boiler Heat [kWh],5044.818728
Total Costs [EUR],4644.296833
Total CO2 [kg],17739.082333


### Compare: What if No CHP?

How much does the CHP save compared to buying all electricity?

In [11]:
# Build system without CHP
fs_no_chp = fx.FlowSystem(timesteps, name='No CHP')
fs_no_chp.add_carriers(
    fx.Carrier('gas', '#3498db', 'kW'),
    fx.Carrier('electricity', '#f1c40f', 'kW'),
    fx.Carrier('heat', '#e74c3c', 'kW'),
)
fs_no_chp.add_elements(
    fx.Bus('Electricity', carrier='electricity'),
    fx.Bus('Heat', carrier='heat'),
    fx.Bus('Gas', carrier='gas'),
    fx.Effect('costs', '€', 'Total Costs', is_standard=True, is_objective=True),
    fx.Effect('CO2', 'kg', 'CO2 Emissions'),
    fx.Source(
        'GasGrid',
        outputs=[fx.Flow('Gas', bus='Gas', size=1000, effects_per_flow_hour={'costs': gas_price, 'CO2': 0.2})],
    ),
    fx.Source(
        'GridBuy',
        outputs=[
            fx.Flow(
                'Electricity', bus='Electricity', size=500, effects_per_flow_hour={'costs': elec_buy_price, 'CO2': 0.4}
            )
        ],
    ),
    # Only boiler for heat
    fx.linear_converters.Boiler(
        'Boiler',
        thermal_efficiency=0.92,
        thermal_flow=fx.Flow('Q_th', bus='Heat', size=500),
        fuel_flow=fx.Flow('Q_fuel', bus='Gas'),
    ),
    fx.Sink(
        'HospitalElec', inputs=[fx.Flow('Load', bus='Electricity', size=1, fixed_relative_profile=electricity_demand)]
    ),
    fx.Sink('HospitalHeat', inputs=[fx.Flow('Load', bus='Heat', size=1, fixed_relative_profile=heat_demand)]),
)

fs_no_chp.optimize(fx.solvers.HighsSolver())

total_costs = flow_system.solution['costs'].item()
total_co2 = flow_system.solution['CO2'].item()
no_chp_costs = fs_no_chp.solution['costs'].item()
no_chp_co2 = fs_no_chp.solution['CO2'].item()

pd.DataFrame(
    {
        'Without CHP': {'Cost [EUR]': no_chp_costs, 'CO2 [kg]': no_chp_co2},
        'With CHP': {'Cost [EUR]': total_costs, 'CO2 [kg]': total_co2},
        'Savings': {
            'Cost [EUR]': no_chp_costs - total_costs,
            'CO2 [kg]': no_chp_co2 - total_co2,
        },
        'Savings [%]': {
            'Cost [EUR]': (no_chp_costs - total_costs) / no_chp_costs * 100,
            'CO2 [kg]': (no_chp_co2 - total_co2) / no_chp_co2 * 100,
        },
    }
)

,Without CHP,With CHP,Savings,Savings [%]
Cost [EUR],11703.962760,4644.296833,7059.665927,60.318595
CO2 [kg],21871.900376,17739.082333,4132.818042,18.895560


### Side-by-Side Comparison

Use the `Comparison` class to visualize both systems together:

In [12]:
comp = fx.Comparison([fs_no_chp, flow_system])
comp.statistics.plot.balance('Electricity')

PlotResult(data=<xarray.Dataset> Size: 12kB
Dimensions:                (case: 2, time: 169)
Coordinates:
  * case                   (case) object 16B 'No CHP' 'With CHP'
  * time                   (time) datetime64[ns] 1kB 2024-02-05 ... 2024-02-12
Data variables:
    GridBuy(Electricity)   (case, time) float64 3kB -140.0 -142.5 ... -41.05 nan
    HospitalElec(Load)     (case, time) float64 3kB 140.0 142.5 ... 161.1 nan
    CHP(P_el)              (case, time) float64 3kB nan nan nan ... -120.0 nan
    GridSell(Electricity)  (case, time) float64 3kB nan nan nan ... 0.0 nan, figure=Figure({
    'data': [{'hovertemplate': ('variable=GridBuy(Electricity)<' ... '}<br>value=%{y}<extra></extra>'),
              'legendgroup': 'GridBuy(Electricity)',
              'marker': {'color': '#636EFA', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'GridBuy(Electricity)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2024-02-05T00:00:00.000000000', '2024-02-05T01:00:00.000000000',
                          '2024-02-05T02:00:00.000000000', '2024-02-05T03:00:00.000000000',
                          '2024-02-05T04:00:00.000000000', '2024-02-05T05:00:00.000000000',
                          '2024-02-05T06:00:00.000000000', '2024-02-05T07:00:00.000000000',
                          '2024-02-05T08:00:00.000000000', '2024-02-05T09:00:00.000000000',
                          '2024-02-05T10:00:00.000000000', '2024-02-05T11:00:00.000000000',
                          '2024-02-05T12:00:00.000000000', '2024-02-05T13:00:00.000000000',
                          '2024-02-05T14:00:00.000000000', '2024-02-05T15:00:00.000000000',
                          '2024-02-05T16:00:00.000000000', '2024-02-05T17:00:00.000000000',
                          '2024-02-05T18:00:00.000000000', '2024-02-05T19:00:00.000000000',
                          '2024-02-05T20:00:00.000000000', '2024-02-05T21:00:00.000000000',
                          '2024-02-05T22:00:00.000000000', '2024-02-05T23:00:00.000000000',
                          '2024-02-06T00:00:00.000000000', '2024-02-06T01:00:00.000000000',
                          '2024-02-06T02:00:00.000000000', '2024-02-06T03:00:00.000000000',
                          '2024-02-06T04:00:00.000000000', '2024-02-06T05:00:00.000000000',
                          '2024-02-06T06:00:00.000000000', '2024-02-06T07:00:00.000000000',
                          '2024-02-06T08:00:00.000000000', '2024-02-06T09:00:00.000000000',
                          '2024-02-06T10:00:00.000000000', '2024-02-06T11:00:00.000000000',
                          '2024-02-06T12:00:00.000000000', '2024-02-06T13:00:00.000000000',
                          '2024-02-06T14:00:00.000000000', '2024-02-06T15:00:00.000000000',
                          '2024-02-06T16:00:00.000000000', '2024-02-06T17:00:00.000000000',
                          '2024-02-06T18:00:00.000000000', '2024-02-06T19:00:00.000000000',
                          '2024-02-06T20:00:00.000000000', '2024-02-06T21:00:00.000000000',
                          '2024-02-06T22:00:00.000000000', '2024-02-06T23:00:00.000000000',
                          '2024-02-07T00:00:00.000000000', '2024-02-07T01:00:00.000000000',
                          '2024-02-07T02:00:00.000000000', '2024-02-07T03:00:00.000000000',
                          '2024-02-07T04:00:00.000000000', '2024-02-07T05:00:00.000000000',
                          '2024-02-07T06:00:00.000000000', '2024-02-07T07:00:00.000000000',
                          '2024-02-07T08:00:00.000000000', '2024-02-07T09:00:00.000000000',
                          '2024-02-07T10:00:00.000000000', '2024-02-07T11:00:00.000000000',
                          '2024-02-07T12:00:00.000000000', '2024-02-07T13:00:00.000000000',
                          '2024-02-07T14:00:00.000000000', '2024-02-07T15:00:00.000000000',
                          '2

In [13]:
comp.statistics.plot.balance('Heat')

PlotResult(data=<xarray.Dataset> Size: 9kB
Dimensions:             (case: 2, time: 169)
Coordinates:
  * case                (case) object 16B 'No CHP' 'With CHP'
  * time                (time) datetime64[ns] 1kB 2024-02-05 ... 2024-02-12
Data variables:
    Boiler(Q_th)        (case, time) float64 3kB -240.5 -222.9 ... -0.0 nan
    HospitalHeat(Load)  (case, time) float64 3kB 240.5 222.9 175.1 ... 150.0 nan
    CHP(Q_th)           (case, time) float64 3kB nan nan nan ... -150.0 nan, figure=Figure({
    'data': [{'hovertemplate': 'variable=Boiler(Q_th)<br>case=No CHP<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Boiler(Q_th)',
              'marker': {'color': '#636EFA', 'line': {'width': 0}, 'pattern': {'shape': ''}},
              'name': 'Boiler(Q_th)',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['2024-02-05T00:00:00.000000000', '2024-02-05T01:00:00.000000000',
                          '2024-02-05T02:00:00.000000000', '2024-02-05T03:00:00.000000000',
                          '2024-02-05T04:00:00.000000000', '2024-02-05T05:00:00.000000000',
                          '2024-02-05T06:00:00.000000000', '2024-02-05T07:00:00.000000000',
                          '2024-02-05T08:00:00.000000000', '2024-02-05T09:00:00.000000000',
                          '2024-02-05T10:00:00.000000000', '2024-02-05T11:00:00.000000000',
                          '2024-02-05T12:00:00.000000000', '2024-02-05T13:00:00.000000000',
                          '2024-02-05T14:00:00.000000000', '2024-02-05T15:00:00.000000000',
                          '2024-02-05T16:00:00.000000000', '2024-02-05T17:00:00.000000000',
                          '2024-02-05T18:00:00.000000000', '2024-02-05T19:00:00.000000000',
                          '2024-02-05T20:00:00.000000000', '2024-02-05T21:00:00.000000000',
                          '2024-02-05T22:00:00.000000000', '2024-02-05T23:00:00.000000000',
                          '2024-02-06T00:00:00.000000000', '2024-02-06T01:00:00.000000000',
                          '2024-02-06T02:00:00.000000000', '2024-02-06T03:00:00.000000000',
                          '2024-02-06T04:00:00.000000000', '2024-02-06T05:00:00.000000000',
                          '2024-02-06T06:00:00.000000000', '2024-02-06T07:00:00.000000000',
                          '2024-02-06T08:00:00.000000000', '2024-02-06T09:00:00.000000000',
                          '2024-02-06T10:00:00.000000000', '2024-02-06T11:00:00.000000000',
                          '2024-02-06T12:00:00.000000000', '2024-02-06T13:00:00.000000000',
                          '2024-02-06T14:00:00.000000000', '2024-02-06T15:00:00.000000000',
                          '2024-02-06T16:00:00.000000000', '2024-02-06T17:00:00.000000000',
                          '2024-02-06T18:00:00.000000000', '2024-02-06T19:00:00.000000000',
                          '2024-02-06T20:00:00.000000000', '2024-02-06T21:00:00.000000000',
                          '2024-02-06T22:00:00.000000000', '2024-02-06T23:00:00.000000000',
                          '2024-02-07T00:00:00.000000000', '2024-02-07T01:00:00.000000000',
                          '2024-02-07T02:00:00.000000000', '2024-02-07T03:00:00.000000000',
                          '2024-02-07T04:00:00.000000000', '2024-02-07T05:00:00.000000000',
                          '2024-02-07T06:00:00.000000000', '2024-02-07T07:00:00.000000000',
                          '2024-02-07T08:00:00.000000000', '2024-02-07T09:00:00.000000000',
                          '2024-02-07T10:00:00.000000000', '2024-02-07T11:00:00.000000000',
                          '2024-02-07T12:00:00.000000000', '2024-02-07T13:00:00.000000000',
                          '2024-02-07T14:00:00.000000000', '2024-02-07T15:00:00.000000000',
                          '2024-02-07T16:00:00.000000000', '2024-02-07T17:00:00.000000000',
                          '2024-02

### Energy Flow Sankey

A Sankey diagram visualizes the total energy flows through the multi-carrier system:

In [14]:
flow_system.statistics.plot.sankey.flows()

PlotResult(data=<xarray.Dataset> Size: 2kB
Dimensions:  (link: 10)
Coordinates:
  * link     (link) int64 80B 0 1 2 3 4 5 6 7 8 9
    source   (link) <U11 440B 'Gas' 'Boiler' 'CHP' ... 'Electricity' 'Heat'
    target   (link) <U12 480B 'Boiler' 'Heat' ... 'HospitalElec' 'HospitalHeat'
    label    (link) <U21 840B 'Boiler(Q_fuel)' ... 'HospitalHeat(Load)'
    carrier  (link) <U11 440B 'gas' 'heat' ... 'electricity' 'heat'
Data variables:
    value    (link) float64 80B 5.483e+03 5.045e+03 ... 3.067e+04 4.418e+04, figure=Figure({
    'data': [{'link': {'color': [rgba(52, 152, 219, 0.4), rgba(231, 76, 60, 0.4),
                                 rgba(241, 196, 15, 0.4), rgba(52, 152, 219, 0.4),
                                 rgba(231, 76, 60, 0.4), rgba(52, 152, 219, 0.4),
                                 rgba(241, 196, 15, 0.4), rgba(241, 196, 15, 0.4),
                                 rgba(241, 196, 15, 0.4), rgba(231, 76, 60, 0.4)],
                       'label': [Boiler(Q_fuel), Boiler(Q_th), CHP(P_el),
                                 CHP(Q_fuel), CHP(Q_th), GasGrid(Gas),
                                 GridBuy(Electricity), GridSell(Electricity),
                                 HospitalElec(Load), HospitalHeat(Load)],
                       'source': [0, 5, 2, 0, 2, 3, 9, 4, 4, 8],
                       'target': [5, 8, 4, 2, 8, 0, 4, 1, 7, 6],
                       'value': [5483.498616953153, 5044.818727596901,
                                 31304.42463983898, 78261.06159959745,
                                 39130.530799798726, 83744.5602165506,
                                 2475.4257247663827, 3108.4415599605,
                                 30671.40880464486, 44175.34952739562]},
              'node': {'color': [#3498db, #00CC96, #AB63FA, #636EFA, #f1c40f,
                                 #FFA15A, #FF6692, #19D3F3, #e74c3c, #EF553B],
                       'label': [Gas, GridSell, CHP, GasGrid, Electricity, Boiler,
                                 HospitalHeat, HospitalElec, Heat, GridBuy],
                       'line': {'color': 'black', 'width': 0.5},
                       'pad': 15,
                       'thickness': 20},
              'type': 'sankey'}],
    'layout': {'template': '...', 'title': {'text': 'Energy Flow'}}
}))

## Key Concepts

### Multi-Carrier Systems

- Multiple buses for different energy carriers (electricity, heat, gas)
- Components can connect to multiple buses (CHP produces both electricity and heat)
- Carriers enable automatic coloring in visualizations

### CHP Modeling

```python
fx.linear_converters.CHP(
    'CHP',
    electrical_efficiency=0.40,  # Fuel → Electricity
    thermal_efficiency=0.50,     # Fuel → Heat
    # Total efficiency = 0.40 + 0.50 = 0.90 (90%)
    electrical_flow=fx.Flow('P_el', bus='Electricity', size=200),
    thermal_flow=fx.Flow('Q_th', bus='Heat', size=250),
    fuel_flow=fx.Flow('Q_fuel', bus='Gas', size=500),
)
```

### Electricity Markets

- **Buy**: Source with positive cost
- **Sell**: Sink with negative cost (= revenue)
- Different prices for buy vs. sell (spread)

### Tracking Multiple Effects

```python
fx.Effect('costs', '€', 'Total Costs', is_objective=True)  # Minimize this
fx.Effect('CO2', 'kg', 'CO2 Emissions')  # Just track, don't optimize
```

## Summary

You learned how to:

- Model **multiple energy carriers** (electricity, heat, gas)
- Use **CHP** for combined heat and power production
- Model **electricity markets** with buy/sell prices
- Track **multiple effects** (costs and emissions)
- Analyze **multi-carrier balances**

### Next Steps

- **[06a-time-varying-parameters](06a-time-varying-parameters.ipynb)**: Variable efficiency based on conditions
- **[07-scenarios-and-periods](07-scenarios-and-periods.ipynb)**: Plan under uncertainty